In [2]:
import rospy
import actionlib

rospy.init_node("body_mover")
print "Node init done"

Node init done


Via Direct trajectory control
------------------------

In [4]:
# Move torso up & arm out of the way
from control_msgs.msg import FollowJointTrajectoryGoal, FollowJointTrajectoryAction
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint

client = actionlib.SimpleActionClient('/arm_with_torso_controller/follow_joint_trajectory', FollowJointTrajectoryAction)
print(client)
client.wait_for_server()
print("Server online")

<actionlib.simple_action_client.SimpleActionClient instance at 0x7fdd6c18b098>
Server online


In [9]:
# Up
joint_names = ["torso_lift_joint", "shoulder_pan_joint",
               "shoulder_lift_joint", "upperarm_roll_joint",
               "elbow_flex_joint", "forearm_roll_joint",
               "wrist_flex_joint", "wrist_roll_joint"]
# Lists of joint angles in the same order as in joint_names
disco_poses = [0.4, 1.32, 0, 0, 0, 0.0, 0, 0.0]
goal = FollowJointTrajectoryGoal()
trajectory = JointTrajectory()
trajectory.joint_names = joint_names
trajectory.points.append(JointTrajectoryPoint())
trajectory.points[0].positions = disco_poses
trajectory.points[0].velocities = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
trajectory.points[0].accelerations = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
trajectory.points[0].time_from_start = rospy.Duration(5.0)
goal.trajectory = trajectory

client.send_goal(goal)
client.wait_for_result(rospy.Duration.from_sec(5.0))

True

Via Planning and Execution
--

In [6]:
from moveit_msgs.msg import MoveItErrorCodes
from moveit_python import MoveGroupInterface, PlanningSceneInterface

# rospy.init_node("move_torso_up")

# Create move group interface for a fetch robot
move_group = MoveGroupInterface("arm_with_torso", "base_link")

# Define ground plane
# This creates objects in the planning scene that mimic the ground
# If these were not in place gripper could hit the ground
planning_scene = PlanningSceneInterface("base_link")
planning_scene.removeCollisionObject("my_front_ground")
planning_scene.removeCollisionObject("my_back_ground")
planning_scene.removeCollisionObject("my_right_ground")
planning_scene.removeCollisionObject("my_left_ground")
planning_scene.addCube("my_front_ground", 2, 1.1, 0.0, -1.0)
planning_scene.addCube("my_back_ground", 2, -1.2, 0.0, -1.0)
planning_scene.addCube("my_left_ground", 2, 0.0, 1.2, -1.0)
planning_scene.addCube("my_right_ground", 2, 0.0, -1.2, -1.0)
# Add table + object --> Hard coded, ! Have to lift arm beforehand, else stuck
planning_scene.removeCollisionObject("table")
planning_scene.addCube("table", 2, 1.1, 0.0, 0.82+0.1-1)

[INFO] [WallTime: 1544049792.854688] [7782.317000] Waiting for get_planning_scene


In [ ]:
# TF joint names
joint_names = ["torso_lift_joint", "shoulder_pan_joint",
               "shoulder_lift_joint", "upperarm_roll_joint",
               "elbow_flex_joint", "forearm_roll_joint",
               "wrist_flex_joint", "wrist_roll_joint"]
# Lists of joint angles in the same order as in joint_names
disco_poses = [[4.0, 1.32, 1.40, -0.2, 1.72, 0.0, 1.66, 0.0]] # Max torso and keep arm out

for pose in disco_poses:
    if rospy.is_shutdown():
        break

    # Plans the joints in joint_names to angles in pose
    move_group.moveToJointPosition(joint_names, pose, wait=False)

    # Since we passed in wait=False above we need to wait here
    move_group.get_move_action().wait_for_result()
    result = move_group.get_move_action().get_result()

    if result:
        # Checking the MoveItErrorCode
        if result.error_code.val == MoveItErrorCodes.SUCCESS:
            rospy.loginfo("Extended Torso.")
        else:
            # If you get to this point please search for:
            # moveit_msgs/MoveItErrorCodes.msg
            rospy.logerr("Arm goal in state: %s",
                         move_group.get_move_action().get_state())
    else:
        rospy.logerr("MoveIt! failure no result returned.")

# This stops all arm movement goals
# It should be called when a program is exiting so movement stops
move_group.get_move_action().cancel_all_goals()

Receive Planned grasp, plan trajectory to it and execute
---

In [3]:
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion
# Receive message from planner
planned_pose = rospy.wait_for_message("/planned_grasp", PoseStamped, timeout=None)
print planned_pose

header: 
  seq: 3
  stamp: 
    secs: 7757
    nsecs: 887000000
  frame_id: base_link
pose: 
  position: 
    x: 0.333381367002
    y: 0.121938478083
    z: 0.819283122622
  orientation: 
    x: 0.572876289032
    y: -0.426284393192
    z: -0.507969358806
    w: -0.48172762439


In [7]:
# Plan and Move to a goal pose
# This is the wrist link not the gripper itself
# The middle between the gripper fingers is 0.16645 m along the x axis
gripper_frame = 'wrist_roll_link'
# Position and rotation of two "wave end poses"
goal_point = planned_pose.pose.position
final_point = Point()
final_point.x = goal_point.x
final_point.y = goal_point.y
final_point.z = goal_point.z
goal_quart = planned_pose.pose.orientation
print "Old goal: \n" + goal_point.__str__()

# want to move in negative x direction as encoded by the quarternion
offset = 0.16645   # Offset between gripper and controled link
distance_approach = 0.3 + offset  # distance from which to start the approach
import tf
matrix = tf.transformations.quaternion_matrix([goal_quart.x,goal_quart.y,goal_quart.z,goal_quart.w])
# print matrix
goal_point.x -= distance_approach*matrix[0][0]
goal_point.y -= distance_approach*matrix[1][0]
goal_point.z -= distance_approach*matrix[2][0]
final_point.x -= offset*matrix[0][0]
final_point.y -= offset*matrix[1][0]
final_point.z -= offset*matrix[2][0]

print "New goal: \n" + goal_point.__str__()
gripper_poses = []
for i in range(10):
    print i
    point = Point()
    point.x = goal_point.x * (9-i)/9 + final_point.x *i/9
    point.y = goal_point.y * (9-i)/9 + final_point.y *i/9
    point.z = goal_point.z * (9-i)/9 + final_point.z *i/9
    print point
    gripper_poses.append(Pose(point,goal_quart))

## For Debug in Rviz
#from std_msgs.msg import Header
#now = rospy.get_rostime()
#hdr = Header(stamp=now, frame_id='base_link')
#pose_msg = PoseStamped(header=hdr, pose=Pose(goal_point, goal_quart))
#print pose_msg
#pub = rospy.Publisher('/test', PoseStamped, queue_size=1, latch=True)
#pub.publish(pose_msg)

Old goal: 
x: 0.333381367002
y: 0.121938478083
z: 0.819283122622
New goal: 
x: 0.277175311265
y: 0.121477017944
z: 1.28233416609
0
x: 0.277175311265
y: 0.121477017944
z: 1.28233416609
1
x: 0.281191894374
y: 0.121509994693
z: 1.24924372814
2
x: 0.285208477483
y: 0.121542971442
z: 1.21615329018
3
x: 0.289225060592
y: 0.121575948191
z: 1.18306285223
4
x: 0.293241643701
y: 0.121608924941
z: 1.14997241428
5
x: 0.29725822681
y: 0.12164190169
z: 1.11688197633
6
x: 0.301274809919
y: 0.121674878439
z: 1.08379153838
7
x: 0.305291393028
y: 0.121707855188
z: 1.05070110043
8
x: 0.309307976138
y: 0.121740831937
z: 1.01761066248
9
x: 0.313324559247
y: 0.121773808686
z: 0.984520224529


In [8]:
# Construct a "pose_stamped" message as required by moveToPose
gripper_pose_stamped = PoseStamped()
gripper_pose_stamped.header.frame_id = 'base_link'

# while not rospy.is_shutdown():
for pose in gripper_poses:
    # Finish building the Pose_stamped message
    # If the message stamp is not current it could be ignored
    gripper_pose_stamped.header.stamp = rospy.Time.now()
    # Set the message pose
    gripper_pose_stamped.pose = pose

    # Move gripper frame to the pose specified
    move_group.moveToPose(gripper_pose_stamped, gripper_frame)
    result = move_group.get_move_action().get_result()
    planning_scene.removeCollisionObject("table") # Only needed for first move

    if result:
        # Checking the MoveItErrorCode
        if result.error_code.val == MoveItErrorCodes.SUCCESS:
            rospy.loginfo("Hello there!")
        else:
            # If you get to this point please search for:
            # moveit_msgs/MoveItErrorCodes.msg
            rospy.logerr("Arm goal in state: %s",
                         move_group.get_move_action().get_state())
    else:
        rospy.logerr("MoveIt! failure no result returned.")

# This stops all arm movement goals
# It should be called when a program is exiting so movement stops
move_group.get_move_action().cancel_all_goals()

[INFO] [WallTime: 1544049811.382135] [7793.742000] Hello there!
[INFO] [WallTime: 1544049812.830312] [7794.718000] Hello there!
[INFO] [WallTime: 1544049814.379908] [7795.718000] Hello there!
[INFO] [WallTime: 1544049815.853443] [7796.739000] Hello there!
[INFO] [WallTime: 1544049817.452260] [7797.749000] Hello there!
[INFO] [WallTime: 1544049819.169407] [7798.744000] Hello there!
[INFO] [WallTime: 1544049820.765163] [7799.754000] Hello there!
[INFO] [WallTime: 1544049822.327051] [7800.750000] Hello there!
[INFO] [WallTime: 1544049824.072953] [7801.744000] Hello there!
[INFO] [WallTime: 1544049826.041543] [7802.951000] Hello there!


[WARN] [WallTime: 1544049880.058372] [7838.113000] Detected jump back in time of 0.001000s. Clearing TF buffer.
